In [1]:
%pip install scikit-learn


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install transformers datasets evaluate accelerate sentencepiece -q
!pip install --upgrade transformers datasets evaluate accelerate -q



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    AutoConfig
)
import evaluate
import numpy as np

/Users/samarveerkapoor/Coding/Projects/PANW-Case-Study/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [6]:

# Load Dataset
print("Loading datasets...")
dataset = load_dataset(
"json",
    data_files={
    "train": ["data/train.jsonl","data/train_v2.jsonl", "data/train_updated.jsonl", "data/idioms.jsonl", "data/greetings.jsonl"],
    "valid": ["data/valid.jsonl", "data/valid_v2.jsonl", "data/idiomsvalid.jsonl", "data/valid-greetings.jsonl"]
}
)
print(f"Train size: {len(dataset['train'])}")
print(f"Valid size: {len(dataset['valid'])}")

Loading datasets...


Generating train split: 4374 examples [00:00, 192129.67 examples/s]
Generating valid split: 540 examples [00:00, 282092.93 examples/s]

Train size: 4374
Valid size: 540


In [7]:
BASE_MODEL = "svkapoor/5EmoteModelRoBERTa"

# Define categories    
label2id = {
    "positive_high_energy": 0,
    "positive_low_energy": 1,
    "negative_high_stress": 2,
    "negative_low_energy": 3,
    "neutral": 4
}
id2label = {v: k for k, v in label2id.items()}
num_labels = len(label2id)

# Tokenize text stream
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

def preprocess(example):
    enc = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    enc["label"] = label2id[example["label"]]
    return enc

print("Tokenizing datasets...")
tokenized = dataset.map(preprocess, batched=False)

Tokenizing datasets...


Map: 100%|██████████| 540/540 [00:00<00:00, 11235.69 examples/s]


In [8]:
# Initialize Model
config = AutoConfig.from_pretrained(
    BASE_MODEL,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)
config.problem_type = "single_label_classification"

model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL,
    config=config,
    ignore_mismatched_sizes=True
)


In [9]:
# Metrics
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return metric.compute(predictions=preds, references=labels)

In [10]:
# Training Arguments
args = TrainingArguments(
    output_dir="./journal_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,
    report_to="none"
)


In [11]:
# Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/var/folders/5w/zbnmt33178b44npdkjvmcqhr0000gp/T/ipykernel_8418/613574069.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
# Train
print("Starting training...")
trainer.train()

Starting training...


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Save model
print("Saving model to ./4emotion_model...")
model.save_pretrained("4emotion_model")
tokenizer.save_pretrained("4emotion_model")
print("Done!")

Saving model to ./4emotion_model...
Done!
